In [1]:
# 시도해봤으나 애매한 것들
# distance, shorest_path_edges_n = brainconn.distance.distance_wei(corr_length) # connection-length matrix, 음수 가중치 없어야 함, (100,100)
# a,b,c = brainconn.distance.distance_wei_floyd(corr) # connection-length matrix, 음수 가중치 사이클이 없어야 함, (100,100)
# characteristic_path_length, global_efficiency, eccentricity, radius, diameter = brainconn.distance.charpath(distance, include_diagonal=True) # distance matrix
# # small-worldness

pip install duecredit

https://sites.google.com/site/bctnet/list-of-measures?authuser=0

In [2]:
!pip install duecredit

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import glob
from pathlib import Path

import brainconn

In [4]:
import os

def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

Networks name : Functional Connectivity (100parcels)

Dataset(Cohert) : UCLA_CNP, COBRE, Decnef

Networks type : undirected, weighted, signed

In [11]:
corr = np.load('data/UCLA_CNP/FC100_sub-10159.npy')
# adj = np.load('data/UCLA_CNP/adjacency/FC100_sub-10159_adj.npy') # node pairwise distance, 0~1사이 값
top_10_percentile_threshold = np.percentile(corr, 90)
fc_matrix_10 = np.where(corr < top_10_percentile_threshold, 0, corr)
fc_matrix_10


# 모든 연결성 값의 절대값을 기준으로 상위 10% 및 20%에 해당하는 임계값 계산
abs_fc_matrix = np.abs(fc_matrix)  # 연결성 값의 절대값
top_10_percentile_threshold = np.percentile(abs_fc_matrix, 90)  # 상위 10%의 임계값
top_20_percentile_threshold = np.percentile(abs_fc_matrix, 80)  # 상위 20%의 임계값

# 원본 연결성 매트릭스에서 상위 10% 및 20% 임계값보다 큰 양수 값과 작은 음수 값을 유지하고 나머지를 0으로 설정
fc_matrix_top_10 = np.where((fc_matrix > top_10_percentile_threshold) | (fc_matrix < -top_10_percentile_threshold), fc_matrix, 0)
fc_matrix_top_20 = np.where((fc_matrix > top_20_percentile_threshold) | (fc_matrix < -top_20_percentile_threshold), fc_matrix, 0)

# 결과 확인
print("상위 10% 임계값:", top_10_percentile_threshold)
print("상위 20% 임계값:", top_20_percentile_threshold)

array([[0.        , 0.        , 0.        , ..., 0.42550347, 0.71972038,
        0.37055566],
       [0.        , 0.        , 0.36731194, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.36731194, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.42550347, 0.        , 0.        , ..., 0.        , 0.47453728,
        0.58119803],
       [0.71972038, 0.        , 0.        , ..., 0.47453728, 0.        ,
        0.52537273],
       [0.37055566, 0.        , 0.        , ..., 0.58119803, 0.52537273,
        0.        ]])

In [86]:
local_measure_dict={
    'betweenness': pd.DataFrame(),
    'eigenvector_centrality': pd.DataFrame(),
    'node_degree': pd.DataFrame(),
    'strength': pd.DataFrame(),
    'strengths_nodal_positive': pd.DataFrame(),
    'strengths_nodal_negative': pd.DataFrame(),
    'clustering_coefficient_positive': pd.DataFrame(),
    'clustering_coefficient_negative': pd.DataFrame(),
    'local_assortativity_positive': pd.DataFrame(),
    'local_assortativity_negative': pd.DataFrame(),
    'clustering_coefficient': pd.DataFrame(),
    'local_efficiency': pd.DataFrame(),
}

global_measure_dict={
    'transitivity': pd.DataFrame(),
    'assortativity': pd.DataFrame(),
    'strengths_total_positive': pd.DataFrame(),
    'strengths_total_negative': pd.DataFrame(),
    'global_efficiency': pd.DataFrame(),
}

In [132]:
class Network(object):
    def __init__(self, corr, adj):
        self.corr = corr
        self.adj = adj

        self.betweenness=None
        self.node_degree=None
        self.strength=None
        self.strengths_nodal_positive=None
        self.strengths_nodal_negative=None
        self.clustering_coefficient=None
        self.clustering_coefficient_positive=None
        self.clustering_coefficient_negative=None
        self.local_assortativity_positive=None
        self.local_assortativity_negative=None
        self.local_efficiency=None
        
        self.transitivity=None
        self.assortativity=None
        self.strengths_total_positive=None
        self.strengths_total_negative=None
        self.global_efficiency=None

    def compute_local_graph_measures(self): #(100,)
        self.betweenness = brainconn.centrality.betweenness_wei(corr)
        self.node_degree = brainconn.degree.degrees_und(corr)
        self.strength = brainconn.degree.strengths_und(corr)
        self.strengths_nodal_positive, self.strengths_nodal_negative,_,_ = brainconn.degree.strengths_und_sign(corr)
        self.clustering_coefficient = brainconn.clustering.clustering_coef_wu(corr)
        self.local_assortativity_positive, self.local_assortativity_negative = brainconn.core.local_assortativity_wu_sign(corr) #invalid value encountered in scalar divide, NAN
        self.clustering_coefficient_positive, self.clustering_coefficient_negative = brainconn.clustering.clustering_coef_wu_sign(corr) 
        self.local_efficiency = brainconn.distance.efficiency_wei(adj, local=True)
        
    def compute_global_graph_measures(self): #(1,)
        self.transitivity = brainconn.clustering.transitivity_wu(corr)
        self.assortativity = brainconn.core.assortativity_wei(corr)
        _,_,self.strengths_total_positive, self.strengths_total_positive = brainconn.degree.strengths_und_sign(corr)
        self.density, vertex_n, edge_n = brainconn.physical_connectivity.density_und(corr)
        self.global_efficiency = brainconn.distance.efficiency_wei(adj, local=False)
        
    def apped_measures_df(self, local_measure_dict, global_measure_dict):
        local_measure_dict['betweenness'] = pd.concat([local_measure_dict['betweenness'], pd.DataFrame(self.betweenness.reshape(-1,100))], ignore_index=True)
        local_measure_dict['node_degree'] = pd.concat([local_measure_dict['node_degree'], pd.DataFrame(self.node_degree.reshape(-1,100))], ignore_index=True)
        local_measure_dict['strength'] = pd.concat([local_measure_dict['strength'], pd.DataFrame(self.strength.reshape(-1,100))], ignore_index=True)
        local_measure_dict['strengths_nodal_positive'] = pd.concat([local_measure_dict['strengths_nodal_positive'], pd.DataFrame(self.strengths_nodal_positive.reshape(-1,100))], ignore_index=True)
        local_measure_dict['strengths_nodal_negative'] = pd.concat([local_measure_dict['strengths_nodal_negative'], pd.DataFrame(self.strengths_nodal_negative.reshape(-1,100))], ignore_index=True)
        local_measure_dict['clustering_coefficient'] = pd.concat([local_measure_dict['clustering_coefficient'], pd.DataFrame(self.clustering_coefficient.reshape(-1,100))], ignore_index=True)
        local_measure_dict['local_assortativity_positive'] = pd.concat([local_measure_dict['local_assortativity_positive'], pd.DataFrame(self.local_assortativity_positive.reshape(-1,100))], ignore_index=True)
        local_measure_dict['local_assortativity_negative'] = pd.concat([local_measure_dict['local_assortativity_negative'], pd.DataFrame(self.local_assortativity_negative.reshape(-1,100))], ignore_index=True)
        local_measure_dict['clustering_coefficient_positive'] = pd.concat([local_measure_dict['clustering_coefficient_positive'], pd.DataFrame(self.clustering_coefficient_positive.reshape(-1,100))], ignore_index=True)
        local_measure_dict['clustering_coefficient_negative'] = pd.concat([local_measure_dict['clustering_coefficient_negative'], pd.DataFrame(self.clustering_coefficient_negative.reshape(-1,100))], ignore_index=True)
        local_measure_dict['local_efficiency'] = pd.concat([local_measure_dict['local_efficiency'], pd.DataFrame(self.local_efficiency.reshape(-1,100))], ignore_index=True)

In [133]:
UCLA_CNP_df=pd.read_csv('data/UCLA_CNP/UCLA_CNP_100FC_phenotype_cognition.csv')[['participant_id', 'diagnosis', 'Cohert', 'PC1']]

In [ ]:
name='local_efficiency'
for subj in UCLA_CNP_df['participant_id']:
    corr = np.load(f'data/UCLA_CNP/FC100_{subj}.npy') # correlation matrix should be zero diagonal
    adj = np.load(f'data/UCLA_CNP/adjacency/FC100_{subj}_adj.npy')
    network = Network(corr, adj)
    network.compute_local_graph_measures()
    network.apped_measures_df(local_measure_dict, global_measure_dict)

In [ ]:
local_measure_dict[name]

In [ ]:
for key in local_measure_dict.keys():
    column_names = [f"{key}_{i+1}" for i in range(100)]
    local_measure_dict[key].columns=column_names
column_names = [f"{name}_{i+1}" for i in range(100)]
local_measure_dict[name].columns=column_names

In [ ]:
local_measure_dict[name]

In [ ]:
for measure_name, measure_df in local_measure_dict.items():
    UCLA_CNP_df = pd.concat([UCLA_CNP_df, measure_df, axis=1)
UCLA_CNP_df = pd.concat([UCLA_CNP_df, local_measure_dict[name]], axis=1)

In [ ]:
UCLA_CNP_df

In [ ]:
for measure in UCLA_CNP_df.columns.to_list()[4:]:
    plt.figure(figsize=(5, 5))
    scatter = plt.scatter(UCLA_CNP_df[measure], UCLA_CNP_df['PC1'])
    
    plt.xlabel(f'{measure}')
    plt.ylabel('Cognition Score')
    dir = f'results/local/{name}'
    createDirectory(dir)
    plt.savefig(Path(dir)/f'{measure}.png')
    plt.show()
    
    plt.close()

In [ ]:
# for measure in UCLA_CNP_df.columns.to_list[4:]
#     plt.figure(figsize=(10, 6))
#     scatter = plt.scatter(UCLA_CNP_df['measure'], UCLA_CNP_df['PC1'], c=UCLA_CNP_df['diagnosis'], cmap='viridis')
    
#     plt.colorbar(scatter)
    
#     plt.xlabel('x')
#     plt.ylabel('y')
#     plt.title('Scatter Plot with Color Representation of z')
#     iloc
#     plt.show()
#     plt.savefig('/mnt/data/scatter_plot.png')

#     plt.close()

In [ ]:
# UCLA_CNP_df['diagnosis'].unique()

In [ ]:
# UCLA_CNP_df['diagnosis'] = UCLA_CNP_df['diagnosis'].map({'CONTROL' : 0, 'SCHZ' : 1, 'BIPOLAR':2, 'ADHD':3})

In [ ]:
# # brainconn에는 있지만 bctpy에는 없는 것
# brainconn.modularity.ci2ls
# brainconn.modularity.ls2ci
# brainconn.modularity.modularity_dir
# brainconn.modularity.modularity_finetune_dir
# brainconn.modularity.modularity_finetune_und
# brainconn.modularity.modularity_finetune_und_sign
# brainconn.modularity.modularity_louvain_dir
# brainconn.modularity.modularity_louvain_und
# brainconn.modularity.modularity_louvain_und_sign
# brainconn.modularity.modularity_probtune_und_sign

# brainconn.modularity.partition_distance
# brainconn.nbs.Doi
# brainconn.nbs.nbs_bct
# brainconn.reference.latmio_dir
# brainconn.reference.latmio_dir_connected
# brainconn.reference.latmio_und
# brainconn.reference.latmio_und_connected
# brainconn.reference.makeevenCIJ
# brainconn.reference.makefractalCIJ
# brainconn.reference.makerandCIJ_und
# brainconn.reference.makerandCIJdegreesfixed
# brainconn.reference.makeringlatticeCIJ
# brainconn.reference.maketoeplitzCIJ
# brainconn.reference.null_model_dir_sign
# brainconn.reference.null_model_und_sign
# brainconn.reference.randmio_dir
# brainconn.reference.randmio_dir_connected
# brainconn.reference.randmio_dir_signed
# brainconn.reference.randmio_und
# brainconn.reference.randmio_und_connected
# brainconn.reference.randmio_und_signed
# brainconn.reference.randomize_graph_partial_und
# brainconn.reference.randomizer_bin_und
# brainconn.similarity.corr_flat_dir
# brainconn.similarity.corr_flat_und
# brainconn.similarity.dice_pairwise_und

# # bctpy에는 있지만 brainconn에는 없는 것
# Diffusion efficiency
# Navigation
# quasi_idempotence


# resilience

# 안 되는 것
#mean_first_passage_time = brainconn.distance.mean_first_passage_time(corr) # (100,100)
#edg_betw_wei = brainconn.centrality.edge_betweenness_wei(corr)[0] #(100, 100)
#community_structure_modularity_link = brainconn.modularity.link_communities(corr) # adjacency matrix, (c, 100)
#rich_club_coefficient = brainconn.core.rich_club_wu(corr) #(99,)
#shortest_path = brainconn.distance.retrieve_shortest_path(corr) # t, hops, pmat 필요
#community_structure_modularity_sign_louvian = brainconn.modularity.community_louvain(corr) # adjacency matrix
#brainconn.modularity.modularity_und_sign(corr) # communitiy partition
# community_structure_modularity_spectral = brainconn.modularity.modularity_und(corr) 
# within_module_degree_zscore_link = brainconn.centrality.module_degree_zscore(corr, community_structure_modularity_link) 
# within_module_degree_zscore_spectral = brainconn.centrality.module_degree_zscore(corr, community_structure_modularity_spectral)
# participation_coef_link = brainconn.centrality.participation_coef(corr, community_structure_modularity_link)
# participation_coef_spectral = brainconn.centrality.participation_coef(corr, community_structure_modularity_spectral)
# participation_coef_sign = brainconn.centrality.participation_coef_sign(corr, community_structure_modularity_sign_louvian)
# participation_coef_sign_gateway = brainconn.centrality.gateway_coef_sign(corr, community_structure_modularity_sign_louvian)
# participation_coef_sign_diversity = brainconn.centrality.diversity_coef_sign(corr, community_structure_modularity_sign_louvian)
# modularity_degeneracy = brainconn.clustering.agreement(corr) # input : partitions of networks
# modularity_degeneracy_wei = brainconn.clustering.agreement_weighted(corr) # input : partitions of networks
# s_core = brainconn.core.score_wu(corr) # s:level of s-core
# consensus_partitioning = brainconn.clustering.consensus_und(corr) # agreement matrix
# self.eigenvector_centrality_pagerank = brainconn.centrality.pagerank_centrality(corr,d=0.85) # adjacency matrix, d:damping factor
# self.eigenvector_centrality = brainconn.centrality.eigenvector_centrality_und(corr) # adjacency matrix

In [ ]:
# UCLACNP_list=[]
# COBRE_list=[]
# Decnef_list=[]
# Total_list=[]

# file_list = glob.glob('data/UCLA_CNP/*.npy')
# for subj in file_list:
#     temp = subj.replace('data/','')
#     temp = temp.replace('.npy', '')
#     UCLACNP_list.append(temp)
    
# file_list = glob.glob('data/COBRE/*.npy')
# for subj in file_list:
#     temp = subj.replace('data/','')
#     temp = temp.replace('.npy', '')
#     COBRE_list.append(temp)
    
# file_list = glob.glob('data/Decnef/*.npy')
# for subj in file_list:
#     temp = subj.replace('data/','')
#     temp = temp.replace('.npy', '')
#     Decnef_list.append(temp)